In [1]:
# %load wavelet.py
import numpy as np
import pywt
import cv2

def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)
    imArray /= 255;
    # compute coefficients
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)
    coeffs_H[0] *= 0;

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)
    print(imArray_H)
    return imArray_H
w2d('C:/Users/SAI PRASATH/Sports celebrity Classification/Model/dataset/cropped/virat_kohli/virat_kohli37.png')

In [3]:
# %load util.py
import joblib
import json
import numpy as np
import base64
import cv2
from wavelet import w2d
import warnings

__class_name_to_number = {}
__class_number_to_name = {}
__model = None

def classify_image(image_base64_data, file_path = None):
    imgs = get_cropped_image_if_2_eyes(file_path, image_base64_data)
    result = []
    for img in imgs:
        scalled_raw_img = cv2.resize(img, (32, 32))
        img_har = w2d(img, 'db1', 5)
        scalled_img_har = cv2.resize(img_har, (32, 32))
        combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1), scalled_img_har.reshape(32*32,1)))
        len_img_arr = 32 * 32 * 3 + 32 * 32
        final = combined_img.reshape(1, len_img_arr).astype(float)
        result.append(__model.predict(final)[0])
        #print(__model.predict(final)[0])
    return __model.predict(final)[0]

def load_artifacts():
    print("Loading saved artifacts...start")
    global __class_name_to_number
    global __class_number_to_name

    with open("C:/Users/SAI PRASATH/Sports celebrity Classification/server/artifacts/final_class.json", "r") as f:
        __class_name_to_number = json.load(f)
        __class_number_to_name = {v:k for k, v in __class_name_to_number.items()}
    global __model
    # __model.__getstate__()['_sklearn_version']
    if __model is None:
        with open('C:/Users/SAI PRASATH/Sports celebrity Classification/server/artifacts/model.pkl', 'rb') as f:
            warnings.simplefilter("ignore", category=UserWarning)
            __model = joblib.load(f)
    print("Loading saved artifacts...done")

def get_cv2_image_from_base64_string(b64str):
    encoded_data = b64str.split(',')[1]
    nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img


def get_cropped_image_if_2_eyes(image_path, image_base64_data):
    face_cascade = cv2.CascadeClassifier('C:/Users/SAI PRASATH/Sports celebrity Classification/server/opencv/haarcascades/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('C:/Users/SAI PRASATH/Sports celebrity Classification/server/opencv/haarcascades/haarcascade_eye.xml')

    if image_path:
        img = cv2.imread(image_path)
    else:
        img = get_cv2_image_from_base64_string(image_base64_data)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    cropped_faces = []
    for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                cropped_faces.append(roi_color)
    return cropped_faces

def get_b64_image():
    with open("C:/Users/SAI PRASATH/Sports celebrity Classification/server/b64_1.txt") as f:
        return f.read()
if __name__ == '__main__':
    load_artifacts()
    print(classify_image(get_b64_image(), None))

Loading saved artifacts...start
Loading saved artifacts...done


error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'
